<a href="https://colab.research.google.com/github/Praneethkumar57/Deeplearning_projs/blob/main/Breast_Tissue_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --quiet

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/tarktunataalt/breast-tissue-impedance-measurements/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: praneethkumar57
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/tarktunataalt/breast-tissue-impedance-measurements


100%|██████████| 8.28k/8.28k [00:00<00:00, 31.7MB/s]

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('/content/breast-tissue-impedance-measurements/data.csv')
df.head()

,Class,I0,PA500,HFS,DA,Area,A.DA,Max.IP,DR,P
0,car,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334
1,car,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776
2,car,551.879287,0.232478,0.063530,264.804935,11888.391827,44.894903,77.793297,253.785300,656.769449
3,car,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701814
4,car,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503


In [ ]:
df.shape

(106, 10)

In [ ]:
df['Class'].value_counts()

,count
Class,
adi,22
car,21
mas,18
gla,16
fad,15
con,14


In [ ]:
df.isnull().sum()

,0
Class,0
I0,0
PA500,0
HFS,0
DA,0
Area,0
A.DA,0
Max.IP,0
DR,0
P,0


In [ ]:
le = LabelEncoder()
df['Target'] = le.fit_transform(df['Class'])
df.head()

,Class,I0,PA500,HFS,DA,Area,A.DA,Max.IP,DR,P,Target
0,car,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334,1
1,car,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776,1
2,car,551.879287,0.232478,0.063530,264.804935,11888.391827,44.894903,77.793297,253.785300,656.769449,1
3,car,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701814,1
4,car,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503,1


In [ ]:
df.drop('Class', axis=1, inplace=True)
df.head()

,I0,PA500,HFS,DA,Area,A.DA,Max.IP,DR,P,Target
0,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334,1
1,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776,1
2,551.879287,0.232478,0.063530,264.804935,11888.391827,44.894903,77.793297,253.785300,656.769449,1
3,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701814,1
4,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503,1


In [ ]:
original_df = df.copy()

In [ ]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
print(X.shape)
print(Y.shape)

(106, 9)
(106,)


In [ ]:
for cols in X.columns:
  X[cols] = X[cols]/X[cols].abs().max()
X.head()

,I0,PA500,HFS,DA,Area,A.DA,Max.IP,DR,P
0,0.187426,0.523137,0.068657,0.215151,0.039223,0.182303,0.138053,0.225806,0.192236
1,0.117857,0.633220,0.567164,0.113927,0.018129,0.159133,0.159866,0.101360,0.138172
2,0.197100,0.648807,0.135821,0.249008,0.068136,0.273630,0.178384,0.259613,0.226739
3,0.135714,0.672187,0.611940,0.129429,0.030961,0.239216,0.203528,0.107614,0.170443
4,0.129583,0.560156,0.522388,0.117461,0.018859,0.160553,0.159114,0.106252,0.146654


In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1)

In [ ]:
X_test,X_val,Y_test,Y_val = train_test_split(X_test,Y_test,test_size=0.5)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(95, 9)
(5, 9)
(6, 9)


In [ ]:
class dataset(Dataset):
    def __init__(self,X,Y):
      self.X = torch.tensor(X,dtype=torch.float32)
      self.Y = torch.tensor(Y,dtype=torch.long)

    def __len__(self):
      return len(self.X)

    def __getitem__(self,index):
      return self.X[index], self.Y[index]


In [ ]:
training_data = dataset(X_train, Y_train)
validation_data = dataset(X_val, Y_val)
testing_data = dataset(X_test, Y_test)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size = 32, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size = 32, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size = 32, shuffle=True)

In [ ]:
for x,y in train_dataloader:
  print(x)
  print("---------------------")
  print(y)
  break

tensor([[5.8929e-01, 1.3298e-01, 9.2537e-02, 2.5805e-01, 3.3384e-02, 1.2937e-01,
         1.8629e-01, 2.6814e-01, 5.5344e-01],
        [4.3214e-02, 4.8612e-01, 1.9403e-01, 2.2979e-02, 8.2798e-04, 3.6032e-02,
         5.0496e-02, 1.0838e-02, 4.8943e-02],
        [7.1429e-01, 1.8753e-01, 2.6567e-01, 3.1057e-01, 8.8154e-02, 2.8385e-01,
         3.8798e-01, 2.9015e-01, 7.1224e-01],
        [7.0130e-02, 5.1047e-01, 3.0448e-01, 5.1327e-02, 4.8330e-03, 9.4161e-02,
         7.8306e-02, 4.3559e-02, 8.2837e-02],
        [5.1429e-02, 3.3658e-01, 9.8507e-02, 1.8476e-02, 4.0363e-04, 2.1847e-02,
         4.1575e-02, 7.7433e-03, 5.5367e-02],
        [3.6786e-02, 4.4131e-01, 6.2388e-01, 2.2338e-02, 4.4852e-04, 2.0079e-02,
         5.1189e-02, 8.3073e-03, 4.3147e-02],
        [1.7929e-01, 1.8217e-01, 5.9701e-02, 5.0063e-02, 4.7815e-03, 9.5508e-02,
         7.6430e-02, 4.2468e-02, 1.8782e-01],
        [9.2816e-02, 1.9727e-01, 1.4925e-02, 5.4769e-02, 2.6656e-03, 4.8669e-02,
         4.0144e-02, 5.7634e-0

In [ ]:
num_classes = len(np.unique(Y))
num_classes

6

In [ ]:
# HIDDEN_NEURONS = 10
# class MyModel(nn.Module):
#   def __init__(self):
#     super(MyModel,self).__init__()

#     self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
#     self.linear = nn.Linear(HIDDEN_NEURONS, 1)
#     self.sigmoid = nn.Sigmoid()

#   def forward(self,x):
#     x = self.input_layer(x)
#     x = self.linear(x)
#     x = self.sigmoid(x)
#     return x

# model = MyModel().to(device)

HIDDEN_NEURONS = 10
#NUM_CLASSES = 3   # example

class MyModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MyModel, self).__init__()

        self.fc1 = nn.Linear(X.shape[1], HIDDEN_NEURONS)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(HIDDEN_NEURONS, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)   # NO sigmoid / softmax here
        return x


In [ ]:
input_dim = X.shape[1]

In [ ]:
model = MyModel(input_dim, num_classes).to(device)

In [ ]:
summary(model,(X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             100
              ReLU-2                   [-1, 10]               0
            Linear-3                    [-1, 6]              66
Total params: 166
Trainable params: 166
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [ ]:
# nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
# total_loss_train_plot = []
# total_loss_validation_plot = []
# total_acc_train_plot = []
# total_acc_validation_plot = []

# epochs = 10

# for epoch in range(epochs):
#   total_acc_train = 0
#   total_loss_train = 0
#   total_acc_val = 0
#   total_loss_val = 0

#   for data in train_dataloader:
#     inputs, labels = data

#     prediction = model(inputs).squeeze(1)

#     batch_loss = criterion(prediction, labels)
#     total_loss_train += batch_loss.item()

#     acc = ((prediction).round() == labels).sum().item()
#     total_acc_train += acc

#     batch_loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

#   with torch.no_grad():
#     for data in validation_dataloader:
#       inputs, labels = data

#       prediction = model(inputs).squeeze(1)

#       batch_loss = criterion(prediction, labels)

#       total_loss_val += batch_loss.item()
#       acc = ((prediction).round() == labels).sum().item()

#       total_acc_val += acc


#   total_loss_train_plot.append(round(total_loss_train/1000,4))
#   total_loss_validation_plot.append(round(total_loss_val/1000,4))

#   total_acc_train_plot.append(round(total_acc_train/training_data.__len__() * 100,4))
#   total_acc_validation_plot.append(round(total_acc_val/validation_data.__len__() * 100,4))

#   print(f'Epoch no. {epoch+1} Train Loss: {round(total_loss_train/1000, 4)} Train Accuracy {round(total_acc_train/training_data.__len__()*100,4)}  Validation Loss: {round(total_loss_val/1000,4)} Validation Accuracy: {round(total_acc_val/validation_data.__len__() * 100,4)}')

#   print("#"*25)

In [ ]:
epochs = 1000

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss_train = 0
    total_acc_train = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)      # shape: (N, C)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()

        preds = torch.argmax(outputs, dim=1)
        total_acc_train += (preds == labels).sum().item()

    train_acc = total_acc_train / len(training_data) * 100
    train_loss = total_loss_train / len(train_dataloader)

    print(f"Epoch {epoch+1} | Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%")


Epoch 1 | Loss: 1.3486 | Acc: 47.37%
Epoch 2 | Loss: 1.3432 | Acc: 47.37%
Epoch 3 | Loss: 1.3373 | Acc: 48.42%
Epoch 4 | Loss: 1.3325 | Acc: 48.42%
Epoch 5 | Loss: 1.3299 | Acc: 48.42%
Epoch 6 | Loss: 1.3224 | Acc: 48.42%
Epoch 7 | Loss: 1.3183 | Acc: 48.42%
Epoch 8 | Loss: 1.3123 | Acc: 49.47%
Epoch 9 | Loss: 1.3082 | Acc: 50.53%
Epoch 10 | Loss: 1.3039 | Acc: 51.58%
Epoch 11 | Loss: 1.3005 | Acc: 51.58%
Epoch 12 | Loss: 1.2959 | Acc: 51.58%
Epoch 13 | Loss: 1.2922 | Acc: 51.58%
Epoch 14 | Loss: 1.2869 | Acc: 51.58%
Epoch 15 | Loss: 1.2833 | Acc: 51.58%
Epoch 16 | Loss: 1.2785 | Acc: 52.63%
Epoch 17 | Loss: 1.2727 | Acc: 52.63%
Epoch 18 | Loss: 1.2702 | Acc: 52.63%
Epoch 19 | Loss: 1.2661 | Acc: 52.63%
Epoch 20 | Loss: 1.2608 | Acc: 52.63%
Epoch 21 | Loss: 1.2586 | Acc: 53.68%
Epoch 22 | Loss: 1.2534 | Acc: 53.68%
Epoch 23 | Loss: 1.2500 | Acc: 53.68%
Epoch 24 | Loss: 1.2463 | Acc: 53.68%
Epoch 25 | Loss: 1.2428 | Acc: 53.68%
Epoch 26 | Loss: 1.2379 | Acc: 54.74%
Epoch 27 | Loss: 1.23

In [ ]:
model.eval()
total_acc_val = 0
total_loss_val = 0

with torch.no_grad():
    for inputs, labels in validation_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        total_loss_val += loss.item()
        preds = torch.argmax(outputs, dim=1)
        total_acc_val += (preds == labels).sum().item()

val_accuracy = total_acc_val / len(validation_data) * 100
val_loss = total_loss_val / len(validation_dataloader)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.2f}%")


Validation Loss: 1.2748
Validation Accuracy: 66.67%
